In [11]:
from bagpy import bagreader
import rospy
import std_msgs.msg
import sensor_msgs.point_cloud2 as pc2

In [74]:
import pandas as pd
import numpy as np
import velodyne_decoder as vd
import open3d as o3d
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time
import pandas as pd

In [12]:
config = vd.Config(model='VLP-16') 
bagfile = '/home/german/Desktop/Project/2017-10-18-17-33-13_0.bag'
lidar_topics = ['/ns1/velodyne_packets'] # '/ns2/velodyne_packets'
cloud_arrays = []
for stamp, points, topic in tqdm(vd.read_bag(bagfile, config, lidar_topics)):
    cloud_arrays.append(points)

0it [00:00, ?it/s]

In [13]:
point_clouds = list(map(lambda x: x[:, :3], cloud_arrays))

In [14]:
def down_segment_cluster(pcd, voxel_size, distance_threshold, ransac_n, num_iterations, eps, min_points):
    
    # Segmentation of the road and objects
    _, inliers = pcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)
    inlier_cloud = pcd.select_by_index(inliers)
    pcd = pcd.select_by_index(inliers, invert=True)
    inlier_cloud.paint_uniform_color([1,0,0])
    pcd.paint_uniform_color([0,0,1])

    # Voxel downsample to remove uneccessary points
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    
    # Clustering and Labeling
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(pcd_down.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))
    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd_down.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    return labels, pcd_down, pcd

In [19]:
# o3d.visualization.draw_geometries([pcd])

In [80]:
params = {'voxel_size': 0.3,
          'distance_threshold': 0.2, 
          'ransac_n': 3, 
          'num_iterations': 500, 
          'eps': 0.9, 
          'min_points': 10}

shorter_frames = list()
for i in tqdm(range(0, 3331, 30)):
    # Create Point Clouds
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_clouds[i])
    # Create Clusters
    labels, pcd_out, pcd = down_segment_cluster(pcd, **params)
    # Append labels and cloud to list
    shorter_frames.append([labels, pcd_out, pcd])
    # break

In [21]:
k = 0
labels = shorter_frames[k][0]
cloud_clusters = shorter_frames[k][1]
original_cloud = shorter_frames[k][2]

In [22]:
o3d.visualization.draw_geometries([cloud_clusters])

In [23]:
def get_bounding_boxes(labels, pcd):
    
    """
    Get the bounding boxes given the labels and the point cloud
    
    """
    
    obbs = []
    indexes = pd.Series(range(len(labels))).groupby(labels, sort=False).apply(list).tolist()

    Max_Points = 1000
    Min_Points = 10

    for i in range(0, len(indexes)):
        nb_pts = len(pcd.select_by_index(indexes[i]).points)
        if (nb_pts > Min_Points and nb_pts < Max_Points):
            sub_cloud = pcd.select_by_index(indexes[i])
            obb = sub_cloud.get_axis_aligned_bounding_box()
            obb.color=(0,0,1)
            obbs.append(obb)
            
    return obbs

In [24]:
bbxs = get_bounding_boxes(labels = labels, pcd = cloud_clusters)
o3d.visualization.draw_geometries([cloud_clusters] + bbxs)

In [14]:
max_clusters = np.array(list(map(lambda x: x[0].max()+1, shorter_frames))).max()
max_clusters

86

In [25]:
def hard_code_add_geometry(vis, bbxs):
    
    lines = [[0, 1], [1, 2], [2, 3], [0, 3], 
             [4, 5], [5, 6], [6, 7], [4, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
    
    for c in range(max_clusters):
        
        line_set = o3d.geometry.LineSet()
        line_set.lines = o3d.utility.Vector2iVector(lines)
        
        if c > len(bbxs) - 1:
            vis.add_geometry(line_set)
            
        else:
            line_set.points = bbxs[c].get_box_points()
            vis.add_geometry(line_set)
    return vis

def hard_code_add_geometry2(vis, bbxs):
    
    lines = [[0, 1], [1, 2], [2, 3], [0, 3], 
             [4, 5], [5, 6], [6, 7], [4, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
    
    for c in range(max_clusters):
        
        vis.clear_geometries()
        
        line_set = o3d.geometry.LineSet()
        line_set.lines = o3d.utility.Vector2iVector(lines)
        
        if c > len(bbxs) - 1:
            # vis.add_geometry(line_set)
            pass
            
        else:
            line_set.points = bbxs[c].get_box_points()
            vis.add_geometry(line_set)
    return vis


def hard_code_update_geometry(vis, new_bbxs):
    
    lines = [[0, 1], [1, 2], [2, 3], [0, 3], 
             [4, 5], [5, 6], [6, 7], [4, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
    
    for c in range(max_clusters):
        
        line_set = o3d.geometry.LineSet()
        line_set.lines = o3d.utility.Vector2iVector(lines)
        
        if c > len(new_bbxs) - 1:
            line_set = line_set.clear()
            vis.update_geometry(line_set)
            
        else:
            line_set = line_set.clear()
            line_set.points = new_bbxs[c].get_box_points()
            vis.update_geometry(line_set)
            
    return vis

In [26]:
lines = [[0, 1], [1, 2], [2, 3], [0, 3], 
         [4, 5], [5, 6], [6, 7], [4, 7],
         [0, 4], [1, 5], [2, 6], [3, 7]]

In [29]:
vis = o3d.visualization.Visualizer()
vis.create_window()

source = o3d.geometry.PointCloud()
source.points = shorter_frames[0][1].points
source.colors = shorter_frames[0][1].colors
vis.add_geometry(source)

bbxs = get_bounding_boxes(shorter_frames[0][0], shorter_frames[0][1])

# vis = hard_code_add_geometry(vis, bbxs)

# line_set = o3d.geometry.LineSet()
# line_set.lines = o3d.utility.Vector2iVector(lines)
# line_set.points = bbxs[10].get_box_points()
# vis.add_geometry(line_set)

# line_set = o3d.geometry.LineSet()
# line_set = line_set.create_from_axis_aligned_bounding_box(bbxs[10])
# vis.add_geometry(line_set)


try:
    for frame_count in range(len(shorter_frames)):
        
        source.points = shorter_frames[frame_count][1].points
        source.colors = shorter_frames[frame_count][1].colors
        vis.update_geometry(source)
        
        new_bbxs = get_bounding_boxes(shorter_frames[frame_count][0], shorter_frames[frame_count][1])
        # vis = hard_code_add_geometry2(vis, bbxs)
        # line_set = line_set.create_from_axis_aligned_bounding_box(new_bbxs[10])
        
        # line_set.lines = o3d.utility.Vector2iVector(lines)
        # line_set.points = new_bbxs[10].get_box_points()
        # vis.update_geometry(line_set)
        
        # vis = hard_code_update_geometry(vis, new_bbxs)
        
        vis.poll_events()
        vis.update_renderer()

        # vis.capture_screen_image("D:\\EECE 5554\\Project\\Imgs\\temp_%05d.jpg" % frame_count)
finally:
    vis.destroy_window()

# Open3d ML

In [65]:
import tensorflow as tf
import open3d.ml.tf as ml3d
# import open3d.ml.torch as ml3d

In [56]:
import os
import open3d.ml as _ml3d

In [49]:
def prepare_point_cloud_for_inference(pcd):
    
    # Remove NaNs and infinity values
    pcd.remove_non_finite_points()
  
    # Extract the xyz points
    xyz = np.asarray(pcd.points)
    
    # Set the points to the correct format for inference
    data = {"point":xyz, 'feat': None, 
            'label':np.zeros((len(xyz),), dtype=np.int32)}
    
    return data, pcd

In [33]:
data, pcd = prepare_point_cloud_for_inference(cloud_clusters)

In [76]:
cfg_file = "Open3D-ML/ml3d/configs/pointpillars_kitti.yml" ################################ FOLDER FROM GITHUB REPOSITORY ######################################################
cfg = _ml3d.utils.Config.load_from_file(cfg_file)
model = ml3d.models.PointPillars(**cfg.model)
pipeline = ml3d.pipelines.ObjectDetection(model, **cfg.pipeline)

In [54]:
# load the parameters (1) ###################################### IF THIS DOESN'T WORK, TRY (2)

ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"
pointpillar_url = "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(pointpillar_url, ckpt_path)
    os.system(cmd)
pipeline.load_ckpt(ckpt_path)

In [78]:
# load the parameters (2)
pipeline.load_ckpt(ckpt_path='/home/german/Desktop/Project/pointpillars_kitti_202012221652utc/ckpt-12.index') ################################ FOLDER FROM DOWNLOADED WEIGHTS ######################################################

0

In [62]:
data

{'point': array([[ 32.37499619, -11.50275421,   5.44173002],
        [ 32.38158798, -11.4732132 ,   9.20515823],
        [ 32.72634125, -11.4862566 ,   4.25860262],
        ...,
        [ 11.91393216,  18.31762505,  -0.38141652],
        [ 14.94410515,  19.95416514,   6.67994722],
        [ 14.08106613,  18.19886716,   1.20592519]]),
 'feat': None,
 'label': array([0, 0, 0, ..., 0, 0, 0], dtype=int32)}

In [79]:
result = pipeline.run_inference(data)

KeyError: 'Exception encountered when calling layer "point_pillars_4" (type PointPillars).\n\n0\n\nCall arguments received:\n  • inputs={\'point\': \'tf.Tensor(shape=(5885, 3), dtype=float32)\', \'feat\': \'None\', \'label\': \'tf.Tensor(shape=(5885,), dtype=int32)\'}\n  • training=False'

# Create Video

In [ ]:
import glob
import cv2
import os

In [ ]:
size = (1920, 1055)
out = cv2.VideoWriter(os.path.join("D:\\EECE 5554\\Project\\", 'Video.mp4'), cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

for pcd_img in tqdm(sorted(glob.glob("D:\\EECE 5554\\Project\\Imgs\\*"))):
    pcd_img = cv2.imread(pcd_img)
    out.write(pcd_img)
out.release()